<a href="https://colab.research.google.com/github/Tharindu021/Context-Based-Spell-Checker/blob/main/create_datataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random
import concurrent.futures

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/Dataset/mispelld sentence and correct sentences/test_data.csv")
input_column = 'input'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def misspell_word(word):
    methods = ['swap', 'replace', 'remove', 'add']
    method = random.choice(methods)

    if method == 'swap' and len(word) > 1:
        idx = random.randint(0, len(word) - 2)
        word = word[:idx] + word[idx+1] + word[idx] + word[idx+2:]

    elif method == 'replace':
        idx = random.randint(0, len(word) - 1)
        random_letter = chr(random.randint(97, 122))
        word = word[:idx] + random_letter + word[idx+1:]

    elif method == 'remove':
        idx = random.randint(0, len(word) - 1)
        word = word[:idx] + word[idx+1:]

    elif method == 'add':
        idx = random.randint(0, len(word))
        random_letter = chr(random.randint(97, 122))
        word = word[:idx] + random_letter + word[idx:]

    return word



In [ ]:
def add_misspelled_word(sentence):
    words = sentence.split()
    if not words:
        return sentence

    valid_words = [i for i, word in enumerate(words) if len(word) >= 3]
    if valid_words:
        idx = random.choice(valid_words)
        words[idx] = misspell_word(words[idx])

    return ' '.join(words)

In [ ]:
def process_batch(batch):
    return batch.apply(lambda x: add_misspelled_word(x) if isinstance(x, str) else x)

def process_in_parallel(df, batch_size=1000):
    batches = [df[input_column][i:i + batch_size] for i in range(0, len(df), batch_size)]

    with concurrent.futures.ProcessPoolExecutor() as executor:
        # Use map to apply the function to each batch in parallel
        results = executor.map(process_batch, batches)

    # Combine all the results back into a single dataframe
    df['misspelled_sentences'] = pd.concat(results, ignore_index=True)
    return df

df['misspelled_sentences'] = df[input_column].apply(add_misspelled_word)

# Process the entire dataset in parallel
df = process_in_parallel(df)

# Save the modified dataset to a new CSV file
df.to_csv('/content/drive/MyDrive/Dataset/mispelld sentence and correct sentences/dataset_with_misspelled_words.csv', index=False)

print("Dataset saved successfully.")


Dataset saved successfully.
